In [1]:
import sys
sys.path.insert(0, '../scripts')

import os
import json
from path import fr1_ctrl_path, fr1_cask_path, fr1_ctrl_sheets, fr1_cask_sheets, \
                rev_ctrl_path, rev_cask_path, rev_ctrl_sheets, rev_cask_sheets
dispense_stats = {}

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from preprocessing import read_excel_by_sheet
from tools import get_bhv_num

def plot_motor_turn(excel_path, sheet):
    data = pd.read_excel(excel_path, sheet_name=sheet)
    data = data[(data['Event'] == 'Pellet') & (data['Motor_Turns'] > 1)][['Motor_Turns', 'Pellet_Count']]
    exceeding_cnt = int((data['Motor_Turns'] > 15).astype(int).sum())
    # group, num = get_bhv_num(sheet)

    # plt.figure(figsize=(15, 5))
    # plt.bar(data['Pellet_Count'], height=data['Motor_Turns'])
    # plt.axhline(y=15, color='r', linestyle='--', linewidth=2)
    # plt.title(f'Motor Turns Statistics of Run {group} Mouse {num}', fontsize=20)
    # plt.xlabel('Pellet Count', fontsize=16)
    # plt.ylabel('Motor Turns', fontsize=16)

    # threshold_line = plt.Line2D([0], [0], color='red', linestyle='--', linewidth=2, label="Threshold")
    # n_count_text = plt.Line2D([0], [0], color='none', label=f"Exceeding Cnt: {exceeding_cnt}")  # Invisible handle

    # plt.legend(handles=[threshold_line, n_count_text])
    # plt.show()
    return exceeding_cnt, round(exceeding_cnt / len(data), 3)

### FR1 ctrl

In [3]:
temp_dict = {}
for sheet in fr1_ctrl_sheets[:]:
    cnt, prop = plot_motor_turn(fr1_ctrl_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['FR1_ctrl'] = temp_dict

In [4]:
temp_dict = {}
for sheet in fr1_cask_sheets:
    cnt, prop = plot_motor_turn(fr1_cask_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['FR1_cask'] = temp_dict

In [5]:
temp_dict = {}
for sheet in rev_ctrl_sheets:
    cnt, prop = plot_motor_turn(rev_ctrl_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['rev_ctrl'] = temp_dict

In [6]:
temp_dict = {}
for sheet in rev_cask_sheets:
    cnt, prop = plot_motor_turn(rev_cask_path, sheet)
    temp_dict[sheet] = [cnt, prop]
dispense_stats['rev_cask'] = temp_dict

In [7]:
with open('../stats/cask_dispense.json', 'w') as f:
    json.dump(dispense_stats, f)

In [8]:
import pickle
import numpy as np
ctrl_acc_dict, cask_acc_dict = pickle.load(open('../data/ctrl_fr1_acc.pkl', 'rb'))
ctrl_dispense_stats, cask_dispense_stats = dispense_stats['FR1_ctrl'], dispense_stats['FR1_cask']

In [9]:
ctrl_array = []
for key, value in ctrl_acc_dict.items():
    temp = [value]
    temp.extend(ctrl_dispense_stats[key])
    ctrl_array.append(temp)

cask_array = []
for key, value in cask_acc_dict.items():
    temp = [value]
    temp.extend(cask_dispense_stats[key])
    cask_array.append(temp)

ctrl_array = np.array(ctrl_array)
cask_array = np.array(cask_array)

In [10]:
print('Control Correlation Matrix')
print(np.corrcoef(ctrl_array, rowvar=False))
print('CASK Correlation Matrix')
print(np.corrcoef(cask_array, rowvar=False))

Control Correlation Matrix
[[1.         0.19210653 0.19212246]
 [0.19210653 1.         0.99982604]
 [0.19212246 0.99982604 1.        ]]
CASK Correlation Matrix
[[1.         0.02985223 0.02985223]
 [0.02985223 1.         1.        ]
 [0.02985223 1.         1.        ]]
